In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import operator
import functools
import math
import os
import skimage.io as io
import random
from scipy import ndimage
import tensorflow as tf
import numpy as np

from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Multiply
from keras.models import Sequential
from keras import backend as K

import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Activation, Input, Lambda, Concatenate,Average, Permute, concatenate
from tensorflow.keras.datasets import mnist,fashion_mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from scipy import ndimage
from tensorflow.keras import activations
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import constraints
import sys
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from random import seed
import cv2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

2023-12-18 18:58:08.600738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-18 18:58:08.600770: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from utils.load_data import *
from utils.Tea import *
from data.preprocess_pma import  *
from modules.baseline import *

# CNN-based extract feature + RANC

In [4]:
def build_model_PMA():

  inputs = Input(shape=(64, 32, 1,))

  x = Conv2D(filters = 8, kernel_size = 3, strides=(1,1), padding = "same")(inputs)
  x = Conv2D(filters = 16, kernel_size = 3, strides=(2,2), padding = "same")(x)
  x = tf.keras.layers.BatchNormalization(axis = 3)(x)
  x = tf.keras.activations.sigmoid(x)
  x = tf.keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same')(x)

  flattened_inputs = Flatten()(x)

  flattened_inputs_1 = Lambda(lambda x : x[:,      :1*2048])(flattened_inputs)

  x1_1  = Lambda(lambda x : x[:,     :256 ])(flattened_inputs_1)
  x2_1  = Lambda(lambda x : x[:, 119 : 375 ])(flattened_inputs_1)
  x3_1  = Lambda(lambda x : x[:, 238 :494 ])(flattened_inputs_1)
  x4_1  = Lambda(lambda x : x[:, 357 : 613])(flattened_inputs_1)
  x5_1  = Lambda(lambda x : x[:, 476:732])(flattened_inputs_1)
  x6_1  = Lambda(lambda x : x[:, 595:851])(flattened_inputs_1)
  x7_1  = Lambda(lambda x : x[:, 714:970])(flattened_inputs_1)
  x8_1  = Lambda(lambda x : x[:, 833:1089])(flattened_inputs_1)
  x9_1  = Lambda(lambda x : x[:, 952:1208])(flattened_inputs_1)
  x10_1  = Lambda(lambda x : x[:, 1071:1327])(flattened_inputs_1)
  x11_1  = Lambda(lambda x : x[:, 1190:1446])(flattened_inputs_1)
  x12_1  = Lambda(lambda x : x[:, 1309:1565])(flattened_inputs_1)
  x13_1  = Lambda(lambda x : x[:, 1428:1684])(flattened_inputs_1)
  x14_1  = Lambda(lambda x : x[:, 1547:1803])(flattened_inputs_1)
  x15_1  = Lambda(lambda x : x[:, 1666:1922])(flattened_inputs_1)
  x16_1  = Lambda(lambda x : x[:, 1785:2041])(flattened_inputs_1)

  x1_1  = Tea(64)(x1_1)
  x2_1  = Tea(64)(x2_1)
  x3_1  = Tea(64)(x3_1)
  x4_1  = Tea(64)(x4_1)
  x5_1  = Tea(64)(x5_1)
  x6_1  = Tea(64)(x6_1)
  x7_1  = Tea(64)(x7_1)
  x8_1  = Tea(64)(x8_1)
  x9_1  = Tea(64)(x9_1)
  x10_1  = Tea(64)(x10_1)
  x11_1  = Tea(64)(x11_1)
  x12_1  = Tea(64)(x12_1)
  x13_1  = Tea(64)(x13_1)
  x14_1  = Tea(64)(x14_1)
  x15_1  = Tea(64)(x15_1)
  x16_1  = Tea(64)(x16_1)

  x1_1 = concatenate(([x1_1,x2_1,x3_1,x4_1]),axis=1)
  x2_1 = concatenate(([x5_1,x6_1,x7_1,x8_1]),axis=1)
  x3_1 = concatenate(([x9_1,x10_1,x11_1,x12_1]),axis=1)
  x4_1 = concatenate(([x13_1,x14_1,x15_1,x16_1]),axis=1)

  x1_1 = Dropout(0.1)(x1_1)
  x2_1 = Dropout(0.1)(x2_1)
  x3_1 = Dropout(0.1)(x3_1)
  x4_1 = Dropout(0.1)(x4_1)

  x1_1 = Tea(64)(x1_1)
  x2_1 = Tea(64)(x2_1)
  x3_1 = Tea(64)(x3_1)
  x4_1 = Tea(64)(x4_1)

  x_out_1 = Concatenate(axis=1)([x1_1,x2_1,x3_1,x4_1])

  x_out_1 = Dropout(0.1)(x_out_1)

  x_out_1 = Tea(255)(x_out_1)

  x_out = AdditivePooling(17)(x_out_1)

  predictions = Activation('softmax')(x_out)

  model1 = Model(inputs=inputs, outputs=predictions)

  return model1

In [ ]:
model = build_model_PMA()
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate = 0.00005),
              metrics=['accuracy'])

In [6]:
## Load data
datasets = load_datasets(path="datasets/experiment-i", type_load="17class", preproc=True)

### Train

In [ ]:
sub = "S1"
x_train, y_train, x_test, y_test = preprocess(datasets=datasets, num_cls=17, sub="S1", hist_equal=True, apply_color_map=False, normalize=True)

checkpoint_filepath = f'/home/luan/works/SNN/checkpoints/PMA_17classes/17_class-{sub}'
model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor = "val_acc",
    save_weights_only=False,
    mode = "max",
    save_best_only = True)
    #initial_value_threshold = 0.8)

In [ ]:
model.fit(x_train, y_train,
          batch_size=256,
          epochs=50,
          verbose=1,
          # callbacks=[model_checkpoint_callback],
          validation_data=(x_train, y_train))

In [ ]:
model.load_weights(checkpoint_filepath)
score = model.evaluate(x_test, y_test, verbose=0)

print('Sub: ', sub)
print('Test loss = ' , score[0])
print('Test acc = ' , score[1])

### Loop training

In [ ]:
for i in range(100):
  print("Training for sub :",sub)

  model.load_weights(checkpoint_filepath)
  score = model.evaluate(x_test, y_test, verbose=0)

  print("Best acc so far: ", score[1])


  model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
      filepath= checkpoint_filepath ,
      monitor = 'val_acc',
      save_best_only = True,
      mode = 'max',
      initial_value_threshold = score[1],
      save_weights_only=False)

  seed()
  lr = 10**(-3*(np.random.rand()) - 2)
  print("Learning rate: ", lr)
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate = lr),
                metrics=['accuracy'])

  model.fit(x_train, y_train,
          batch_size=64,
          epochs=50,
          verbose=1,
          callbacks=[model_checkpoint_callback],
          validation_data = (x_test,y_test))

In [ ]:
sub_train = ["S4","S5"]
subjects = ["S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12","S13"]

for sub in sub_train:
  checkpoint_filepath = f'/home/luan/works/SNN/checkpoints/PMA_17classes/17_class-{sub}'

  x_train, y_train, x_test, y_test = preprocess(datasets=datasets, num_cls=17, sub=sub, hist_equal=True, apply_color_map=False, normalize=True)

  model = build_model_PMA()
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate = 0.0004),
                metrics=['accuracy'])


  model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      monitor = "val_acc",
      save_weights_only=False,
      mode = "max",
      save_best_only = True)

  if not os.path.exists(checkpoint_filepath):
    model.fit(x_train, y_train,
              batch_size=32,
              epochs=50,
              verbose=1,
              callbacks=[model_checkpoint_callback],
              validation_data=(x_test, y_test))
  else:
    model.load_weights(checkpoint_filepath)
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Best Accuracy so far: ", score[1]*100)

    if score[1] < 0.9:
      print("RETRAINING..................")
      model = build_model_PMA()
      seed()
      lr = 10**(-3*(np.random.rand()) - 3)

      model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
          filepath=checkpoint_filepath,
          monitor = "val_acc",
          save_weights_only=False,
          mode = "max",
          save_best_only = True
          initial_value_threshold = score[1])
      
      model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(learning_rate = lr),
                    metrics=['accuracy'])

      model.fit(x_train, y_train,
              batch_size=32,
              epochs=30,
              verbose=1,
              callbacks=[model_checkpoint_callback],
              validation_data = (x_test,y_test))




  for i in range(100):
    print("Training for sub :",sub)


    model.load_weights(checkpoint_filepath)
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Best Accuracy so far: ", score[1]*100)

    model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
        filepath= checkpoint_filepath ,
        monitor = 'val_acc',
        save_best_only = True,
        mode = 'max',
        initial_value_threshold = score[1],
        save_weights_only=False)

    seed()
    lr = 10**(-3*(np.random.rand()) - 4)
    print("Learning rate: ", lr)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate = lr),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
            batch_size=32,
            epochs=30,
            verbose=1,
            callbacks=[model_checkpoint_callback],
            validation_data = (x_test,y_test))

In [ ]:
checkpoints = os.listdir('/home/luan/works/SNN/checkpoints/PMA_17classes')

for ckpt in checkpoints:

  sub = ckpt.split("-")[1]
  x_train, y_train, x_test, y_test = preprocess(datasets=datasets, num_cls=17, sub=sub, hist_equal=True, apply_color_map=False, normalize=True)
  model = build_model_PMA()

  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate = 0.00005),
                metrics=['accuracy'])

  model.load_weights(f'/home/luan/works/SNN/checkpoints/PMA_17classes/17_class-{sub}')
  score = model.evaluate(x_test, y_test, verbose=0)

  print('Sub: ', sub)
  print('Test loss = ' , score[0])
  print('Test acc = ' , score[1])
